# CloudML with DataLAB Part. 5

### Distributed Learning

Hello!

Thanks for comming again!

This session, we would learn how to train the model as distributed learning

Seems so difficault, actually not

Let's go with simple code at first

In [34]:
import tensorflow as tf

c = tf.constant("Hello, distributed TensorFlow!")
server = tf.train.Server.create_local_server()

# create a session on the server
with tf.Session(server.target) as sess:
    print(sess.run(c))

Hello, distributed TensorFlow!


Distributed learning means that model is trained by the distributed computing

Distrubute means the many of computers for training exist

Yeah, right! That's why the server is needed.

During training, many of computers interact each other to recieve the data to calculate

So I created a server and opened the session in the server.target This is the simplest code of distributed learning

In this case, the local computer (this instance) runs only.  If I have a CPU and a GPU, I just can use resource from them. only them!

Well... is that what you want?

Let's use more as we can! check out the code below

In [33]:
%%writefile ./model/test.py
import os
import json
from pprint import pprint

# The environment variable "TF_CONFIG" will be defined when the ml-engine runs
# but returns None before 
tf_config = os.environ.get('TF_CONFIG')

# you can read it as json.
tf_config_json = json.loads(tf_config)

# We need the cluster spec to create distributed server
# What we get like below is included in the spec
cluster = tf_config_json.get('cluster')
job_name = tf_config_json.get('task', {}).get('type')
task_index = tf_config_json.get('task', {}).get('index')

# Let's see what there is
pprint(tf_config_json)
print "=========================================================="

Overwriting ./model/test.py


Of course there is nothing to see if you run this code by python interpreter

Set the server count, worker count and whether distributed then run the ml-engine so TF_CONFIG would be set

Now you can see the result like below

In [36]:
%bash
gcloud ml-engine local train \
--package-path=$(pwd)/model \
--module-name=model.test \
--parameter-server-count=2 \
--worker-count=3 \
--distributed

{u'cluster': {u'master': [u'https://27182-dot-2183444-dot-devshell.appspot.com'],
              u'ps': [u'https://27183-dot-2183444-dot-devshell.appspot.com', u'https://27184-dot-2183444-dot-devshell.appspot.com'],
              u'worker': [u'https://27185-dot-2183444-dot-devshell.appspot.com',
                          u'https://27186-dot-2183444-dot-devshell.appspot.com',
                          u'https://27187-dot-2183444-dot-devshell.appspot.com']},
 u'environment': u'cloud',
 u'job': {u'args': [], u'job_name': u'model.test'},
 u'task': {u'index': 2, u'type': u'worker'}}
{u'cluster': {u'master': [u'https://27182-dot-2183444-dot-devshell.appspot.com'],
              u'ps': [u'https://27183-dot-2183444-dot-devshell.appspot.com', u'https://27184-dot-2183444-dot-devshell.appspot.com'],
              u'worker': [u'https://27185-dot-2183444-dot-devshell.appspot.com',
                          u'https://27186-dot-2183444-dot-devshell.appspot.com',
                          u'https://271

You could be curious what worker is or ps is.

just take it easy to say that ps is CPU and worker is GPU

ps is for operation of model, worker is for calculate

Details are in video [here](https://youtu.be/la_M6bCV91M)

If you got understand what those is, check out the "task" in the log,

It has the values "index" "type". you can see how many worker are(actually they're that you set) or how many ps are

ps is same as server count, worker count is variable.

And also you can see the type "master". It's to be a main server that you specified to run the model

Now we would see how to construct the server to distributed learning

Let's build the model to test first

In [5]:
%%writefile ./model/test.py
import tensorflow as tf
import os
import json
from pprint import pprint

# get environment variable TF_CONFIG to specify the cluster
tf_config = os.environ.get('TF_CONFIG')

tf_config_json = json.loads(tf_config)

cluster = tf_config_json.get('cluster')
job_name = tf_config_json.get('task', {}).get('type')
task_index = tf_config_json.get('task', {}).get('index')

# This is how to create server for distrubted learning
cluster_spec = tf.train.ClusterSpec(cluster)
server = tf.train.Server(cluster_spec,
                         job_name=job_name,
                         task_index=task_index)

# this model is built last session
data = [[i for i in range(10)], [9-i for i in range(10)]]
label = [[1], [0]]
x = tf.placeholder(tf.float32, [None, 10])
y_ = tf.placeholder(tf.float32, [None, 1])
feed_dict = {x: data, y_: label}

layer1 = tf.layers.dense(x, 10)
layer2 = tf.layers.dense(layer1, 10)
layer3 = tf.layers.dense(layer2, 10)
logits = tf.layers.dense(layer3, 1)

loss = tf.losses.sigmoid_cross_entropy(y_, logits)
train = tf.train.GradientDescentOptimizer(0.01).minimize(loss)

# Run distributed learning!
with tf.Session(server.target) as sess:
    sess.run(tf.global_variables_initializer())
    for i in range(100):
        _, _loss = sess.run([train, loss], feed_dict=feed_dict)
        if i % 10 == 0:
            print("loss: ", _loss)

Overwriting ./model/test.py


In [8]:
%bash
gcloud ml-engine local train \
--package-path=$(pwd)/model \
--module-name=model.test \
--parameter-server-count=2 \
--worker-count=3 \
--distributed

('loss: ', 0.50345963)
('loss: ', 0.00014202719)
('loss: ', 0.00011008905)
('loss: ', 9.104309e-05)
('loss: ', 7.7581746e-05)
('loss: ', 6.7566274e-05)
('loss: ', 6.0151204e-05)
('loss: ', 5.4369248e-05)
('loss: ', 5.042235e-05)
('loss: ', 4.6871926e-05)
('loss: ', 0.50345963)
('loss: ', 0.00014632563)
('loss: ', 0.00011341777)
('loss: ', 9.2545917e-05)
('loss: ', 7.904564e-05)
('loss: ', 6.7839814e-05)
('loss: ', 5.9507645e-05)
('loss: ', 5.4281023e-05)
('loss: ', 4.9818937e-05)
('loss: ', 4.6092333e-05)
('loss: ', 0.50345963)
('loss: ', 0.00014262296)
('loss: ', 0.00011228817)
('loss: ', 9.3577437e-05)
('loss: ', 7.7760989e-05)
('loss: ', 6.7566274e-05)
('loss: ', 5.9507645e-05)
('loss: ', 5.4192737e-05)
('loss: ', 5.027006e-05)
('loss: ', 4.6478945e-05)
('loss: ', 0.50345963)
('loss: ', 0.00014632563)
('loss: ', 0.00011615325)
('loss: ', 9.4101364e-05)
('loss: ', 7.904564e-05)
('loss: ', 6.839422e-05)
('loss: ', 6.0368478e-05)
('loss: ', 5.463645e-05)
('loss: ', 5.027006e-05)
('loss

W tensorflow/core/platform/cpu_feature_guard.cc:45] The TensorFlow library wasn't compiled to use SSE3 instructions, but these are available on your machine and could speed up CPU computations.
W tensorflow/core/platform/cpu_feature_guard.cc:45] The TensorFlow library wasn't compiled to use SSE4.1 instructions, but these are available on your machine and could speed up CPU computations.
W tensorflow/core/platform/cpu_feature_guard.cc:45] The TensorFlow library wasn't compiled to use SSE4.2 instructions, but these are available on your machine and could speed up CPU computations.
W tensorflow/core/platform/cpu_feature_guard.cc:45] The TensorFlow library wasn't compiled to use AVX instructions, but these are available on your machine and could speed up CPU computations.
W tensorflow/core/platform/cpu_feature_guard.cc:45] The TensorFlow library wasn't compiled to use AVX2 instructions, but these are available on your machine and could speed up CPU computations.
W tensorflow/core/platform/

It's not the end. distributed learning is run but it's meaningless because model was run at each server as same

We need to saperate each variables or operation to each server

To do that, all the ps nodes have to be set to listen data for operate and all workers have to be set to execute the calculation 

There might be few changes in the session also. check this out

In [27]:
%%writefile ./model/test.py
import tensorflow as tf
import os
import json
from pprint import pprint

# get environment variable TF_CONFIG to specify the cluster
tf_config = os.environ.get('TF_CONFIG')

tf_config_json = json.loads(tf_config)

cluster = tf_config_json.get('cluster')
job_name = tf_config_json.get('task', {}).get('type')
task_index = tf_config_json.get('task', {}).get('index')


# This is how to create server for distrubted learning
cluster_spec = tf.train.ClusterSpec(cluster)
server = tf.train.Server(cluster_spec,
                         job_name=job_name,
                         task_index=task_index)

data = [[i for i in range(10)], [9-i for i in range(10)]]
label = [[1], [0]]

# set ps listening
if job_name == 'ps':
  server.join()
elif job_name == 'worker':
  # seperate variables to the workers
  with tf.device(tf.train.replica_device_setter()):
    x = tf.placeholder(tf.float32, [None, 10])
    y_ = tf.placeholder(tf.float32, [None, 1])
    feed_dict = {x: data, y_: label}
    
    layer1 = tf.layers.dense(x, 10)
    layer2 = tf.layers.dense(layer1, 10)
    layer3 = tf.layers.dense(layer2, 10)
    logits = tf.layers.dense(layer3, 1)
    
    # you cannot use tf.Session() to train the model using for .. in ... 
    # so you need to create the object which define the global step
    global_step = tf.contrib.framework.get_or_create_global_step()
    
    loss = tf.losses.sigmoid_cross_entropy(y_, logits)
    train = tf.train.GradientDescentOptimizer(0.01).minimize(loss, global_step=global_step)

  # you can set the global step here
  hooks=[tf.train.StopAtStepHook(last_step=100)]
  
  # new session to distributed learning!
  # use this session following this code
  with tf.train.MonitoredTrainingSession(master=server.target,
                                         is_chief=(task_index == 0),
                                         checkpoint_dir="./model/train_logs",
                                         hooks=hooks) as mon_sess:
    while not mon_sess.should_stop():
      # Run a training step asynchronously.
      # See `tf.train.SyncReplicasOptimizer` for additional details on how to
      # perform *synchronous* training.
      # mon_sess.run handles AbortedError in case of preempted PS.
      mon_sess.run(train)

Overwriting ./model/test.py


In [28]:
%bash
gcloud ml-engine local train \
--package-path=$(pwd)/model \
--module-name=model.test \
--parameter-server-count=2 \
--worker-count=3 \
--distributed

W tensorflow/core/platform/cpu_feature_guard.cc:45] The TensorFlow library wasn't compiled to use SSE3 instructions, but these are available on your machine and could speed up CPU computations.
W tensorflow/core/platform/cpu_feature_guard.cc:45] The TensorFlow library wasn't compiled to use SSE3 instructions, but these are available on your machine and could speed up CPU computations.
W tensorflow/core/platform/cpu_feature_guard.cc:45] The TensorFlow library wasn't compiled to use SSE4.1 instructions, but these are available on your machine and could speed up CPU computations.
W tensorflow/core/platform/cpu_feature_guard.cc:45] The TensorFlow library wasn't compiled to use SSE4.2 instructions, but these are available on your machine and could speed up CPU computations.
W tensorflow/core/platform/cpu_feature_guard.cc:45] The TensorFlow library wasn't compiled to use AVX instructions, but these are available on your machine and could speed up CPU computations.
W tensorflow/core/platform/

You got it if you see the message like this

When you want to run the ml-engine as JOB, just set the SCALE_TIER to STANDARD_1

The others are same as we've done

In [30]:
%bash
PROJECT_ID=tensorflowprojects
BUCKET="gs://${PROJECT_ID}-mlengine"
JOB_NAME=simple_model_$(date +%Y%m%d_%H%M%S)
PACKAGE_PATH=$(pwd)/model
MODULE_NAME=model.tast
STAGING_BUCKET=${BUCKET}
JOB_DIR=${BUCKET}/${JOB_NAME}
OUTPUT=${BUCKET}/${JOB_NAME}
REGION=europe-west1
SCALE_TIER=STANDARD_1

# Submit job with these settings
gcloud ml-engine jobs submit training $JOB_NAME \
--package-path=$PACKAGE_PATH \
--module-name=$MODULE_NAME \
--staging-bucket=$STAGING_BUCKET \
--job-dir=$JOB_DIR \
--scale-tier=$SCALE_TIER \
--region=$REGION 

jobId: simple_model_20170612_090123
state: QUEUED


Job [simple_model_20170612_090123] submitted successfully.
Your job is still active. You may view the status of your job with the command

  $ gcloud ml-engine jobs describe simple_model_20170612_090123

or continue streaming the logs with the command

  $ gcloud ml-engine jobs stream-logs simple_model_20170612_090123


Well, It seems most of necessary things have been written

More details about this are in the [documents](https://www.tensorflow.org/deploy/distributed)

Thanks for reading!